In [23]:
import os
import re
import sys
import json
import time
import math
import requests
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
from rapidfuzz import process, fuzz
from tqdm import tqdm

# ===== 모듈 경로 추가 & 로거 설정 =====
sys.path.append(r"C:\ESG_Project1\util")
from logger import setup_logger
logger = setup_logger(__name__)

# ----------------------------------------------------
# 🔹 경로 설정
# ----------------------------------------------------
BASE_DIR = "C:/ESG_Project1/file/"
DATA_DIR = "C:/ESG_Project1/data/"
SOLAR_DIR = os.path.join(BASE_DIR, "solar_data_file")
KMA_DIR = os.path.join(BASE_DIR, "KMA_data_file")
OUT_CSV = os.path.join(BASE_DIR, "merge_data", "train_data.csv")

WEATHER_META = os.path.join(KMA_DIR, "META_관측지점정보.csv")
CACHE_JSON = os.path.join(DATA_DIR, "json/mapping_cache.json")
REGION_FIX_JSON = os.path.join(DATA_DIR, "json/region_fix.json")
GEO_JSON = os.path.join(DATA_DIR, "json/namdong_geo.json")

# ----------------------------------------------------
# 🔹 CSV 유틸
# ----------------------------------------------------
def sniff_delimiter(path):
    with open(path,"rb") as f:
        raw=f.read(2048)
    text=raw.decode("utf-8",errors="ignore")
    return "," if text.count(",") >= text.count("\t") else "\t"

def read_csv_safe(path):
    delim = sniff_delimiter(path)
    try:
        df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
        logger.info(f"📄 '{os.path.basename(path)}' 읽기 성공 (utf-8)")
        return df
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp949", delimiter=delim, index_col=False)
        logger.info(f"📄 '{os.path.basename(path)}' 읽기 성공 (cp949)")
        return df

def collect_files():
    files=[]
    folder=os.path.join(SOLAR_DIR)
    files+=glob(os.path.join(folder,"20*/*.csv")) + glob(os.path.join(folder,"20*/*.CSV"))
    logger.info(f"🗂 {len(files)}개의 발전소 CSV 파일 수집됨")
    return files

def normalize_columns(df):
    df.columns=df.columns.str.strip()
    if "발전구분" not in df.columns:
        expected = ["발전구분","호기","일자"] + [f"{i}시 발전량(MWh)" for i in range(1,25)]
        df=df.iloc[:,:len(expected)]
        df.columns=expected
    df["발전구분"]=df["발전구분"].astype(str).str.strip()
    df["일자"]=pd.to_datetime(df["일자"],errors="coerce")
    return df

def get_hour_cols(df):
    pat = re.compile(r"^\s*(\d{1,2})시")
    return [c for c in df.columns if pat.search(c)]

# ----------------------------------------------------
# 🔹 1. 남동발전 발전소 매핑 크롤러
# ----------------------------------------------------
def crawl_mapping():
    URL = "https://www.koenergy.kr/kosep/hw/fr/ov/ovhw25/main.do?menuCd=FN060202"
    logger.info(f"🌐 크롤링 시작: {URL}")
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(URL, headers=headers, timeout=10)
    res.raise_for_status()
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    tables = soup.find_all("table", class_="table_list2")
    if len(tables) < 2:
        raise RuntimeError("❌ class='table_list2' 테이블이 2개 이상 존재하지 않습니다.")
    table = tables[1]
    mapping = {}
    for tr in table.find_all("tr"):
        tds = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(tds) < 2: continue
        사업명, 지역 = tds[0], tds[1]
        if "태양광" not in 사업명: continue
        발전구분 = 사업명.replace("발전소","").replace(" ","").strip()
        mapping[발전구분] = 지역
    os.makedirs(os.path.dirname(CACHE_JSON), exist_ok=True)
    with open(CACHE_JSON,"w",encoding="utf-8") as f: json.dump(mapping,f,ensure_ascii=False,indent=2)
    logger.info(f"✅ {len(mapping)}개 항목 크롤링 완료 → {CACHE_JSON}")
    return mapping

# ----------------------------------------------------
# 🔹 2. CSV 통합 및 호기 합산
# ----------------------------------------------------
files = collect_files()
if not files:
    logger.error("⚠️ CSV 파일이 없습니다.")
    sys.exit(1)

frames=[]
for f in tqdm(files, desc="CSV 불러오기"):
    try:
        tmp = read_csv_safe(f)
        tmp = normalize_columns(tmp)
        frames.append(tmp)
        logger.info(f"📄 {os.path.basename(f)} 불러옴 ({len(tmp)}행)")
        logger.info(f"   컬럼: {tmp.columns.tolist()}")
        logger.info(f"   샘플:\n{tmp.head(2)}")
    except Exception as e:
        logger.warning(f"⚠️ {os.path.basename(f)} 읽기 실패: {e}")

df = pd.concat(frames,ignore_index=True).drop_duplicates()
logger.info(f"🧹 완전 동일행 중복 제거: {len(df)}행")

hour_cols=get_hour_cols(df)
long_df=df.melt(id_vars=["발전구분","호기","일자"],
                value_vars=hour_cols,
                var_name="시간대",
                value_name="발전량(MWh)")
long_df["시간"]=long_df["시간대"].str.extract(r"(\d{1,2})").astype(int)
long_df["일시"]=pd.to_datetime(long_df["일자"],errors="coerce")+pd.to_timedelta(long_df["시간"]-1,"h")

hoqi_sum=long_df.groupby(["발전구분","일자","시간"],as_index=False)["발전량(MWh)"].sum(numeric_only=True).rename(columns={"발전량(MWh)":"합산발전량(MWh)"})
hoqi_sum["일시"]=pd.to_datetime(hoqi_sum["일자"],errors="coerce")+pd.to_timedelta(hoqi_sum["시간"]-1,"h")
hoqi_sum=hoqi_sum[["일시","발전구분","합산발전량(MWh)"]]
logger.info(f"📊 호기 합산 완료: {len(hoqi_sum)}행")
logger.info(f"   샘플:\n{hoqi_sum.head(3)}")

# ----------------------------------------------------
# 🔹 3. 발전소명 → 지역 매핑 + 보정
# ----------------------------------------------------
if os.path.exists(CACHE_JSON):
    with open(CACHE_JSON,"r",encoding="utf-8") as f: mapping=json.load(f)
    logger.info(f"📦 캐시 사용 → {CACHE_JSON}")
else:
    mapping=crawl_mapping()

if not os.path.exists(REGION_FIX_JSON):
    default_region_fix = {"영흥":"인천광역시 옹진군 영흥면","삼천포":"경상남도 고성군","고성":"경상남도 고성군",
                         "예천":"경상북도 예천군","여수":"전라남도 여수시","영동":"강원특별자치도 강릉시",
                         "구미":"경상북도 구미시","장성":"전라남도 장성군","진주":"경상남도 진주시",
                         "광양":"전라남도 광양시","창원":"경상남도 창원시 마산합포구 해운동","고흥":"전라남도 고흥군",
                         "군산":"전라북도 군산시","밀양":"경상남도 밀양시","서산":"충청남도 서산시","영암":"전라남도 영암군",
                         "신안":"전라남도 신안군","강릉":"강원특별자치도 강릉시","전국":"대한민국","진주 외":"경상남도 진주시 외"}
    os.makedirs(os.path.dirname(REGION_FIX_JSON), exist_ok=True)
    with open(REGION_FIX_JSON,"w",encoding="utf-8") as f: json.dump(default_region_fix,f,ensure_ascii=False,indent=2)
    logger.info(f"🆕 지역 보정 테이블 생성 → {REGION_FIX_JSON}")

with open(REGION_FIX_JSON,"r",encoding="utf-8") as f: region_fix=json.load(f)

def normalize_name(name):
    name=re.sub(r"\s+","",str(name))
    name=name.replace("발전소","").replace("태양광","").replace("-","").replace("_","").strip()
    return name

normalized_mapping={normalize_name(k):v for k,v in mapping.items()}
hoqi_sum["발전구분_정규화"]=hoqi_sum["발전구분"].apply(normalize_name)

keys=list(normalized_mapping.keys())
match_cache={}
def fast_match(name):
    if name in match_cache: return match_cache[name]
    for key in keys:
        if key in name or name in key:
            result=normalized_mapping[key]
            if result in region_fix: result=region_fix[result]
            match_cache[name]=result
            return result
    match=process.extractOne(name, keys, scorer=fuzz.partial_ratio)
    if match and match[1]>=75:
        result=normalized_mapping[match[0]]
        if result in region_fix: result=region_fix[result]
        match_cache[name]=result
        return result
    match_cache[name]=None
    return None

mapping_result={n:fast_match(n) for n in tqdm(hoqi_sum["발전구분_정규화"].unique(), desc="지역 매핑")}
hoqi_sum["지역"]=hoqi_sum["발전구분_정규화"].map(mapping_result)
logger.info(f"📌 지역 매핑 완료")
logger.info(f"   샘플:\n{hoqi_sum.head(3)}")

# ----------------------------------------------------
# 🔹 4. 카카오 API → 좌표 + 최근접 기상관측소
# ----------------------------------------------------
KAKAO_API_KEY = "93c089f75a2730af2f15c01838e892d3"
KAKAO_URL = "https://dapi.kakao.com/v2/local/search/address.json"

def get_latlng(address):
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    try:
        res = requests.get(KAKAO_URL, headers=headers, params=params, timeout=5)
        res.raise_for_status()
        docs = res.json().get("documents")
        if docs: 
            return float(docs[0]["y"]), float(docs[0]["x"])
        return None, None
    except Exception as e:
        logger.warning(f"⚠️ 주소 '{address}' 조회 실패: {e}")
        return None, None

# 기상 관측소 정보
weather_df = read_csv_safe(WEATHER_META)
weather_df = weather_df.rename(columns=str.strip)[["지점","지점명","위도","경도"]].dropna(subset=["위도","경도"])

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(a))

def find_nearest_station(lat, lon, meta_df):
    meta_df["거리(km)"] = meta_df.apply(lambda r: haversine(lat, lon, r["위도"], r["경도"]), axis=1)
    nearest = meta_df.loc[meta_df["거리(km)"].idxmin()]
    return {"지점명": nearest["지점명"], "지점번호": int(nearest["지점"]),
            "위도": nearest["위도"], "경도": nearest["경도"], "거리(km)": round(nearest["거리(km)"],2)}

# GEO_JSON 캐시 사용
if not os.path.exists(GEO_JSON):
    logger.info("🌍 좌표+최근접 관측소 매핑 시작")
    geo_cache = {}
    for region in tqdm(sorted(set(v for v in hoqi_sum["지역"] if v)), desc="좌표 조회"):
        lat, lng = get_latlng(region)
        if lat is None or lng is None:
            geo_cache[region] = {"위도": None, "경도": None, "최근접관측소": None}
            continue
        nearest = find_nearest_station(lat, lng, weather_df)
        geo_cache[region] = {"위도": lat, "경도": lng, "최근접관측소": nearest}
        time.sleep(0.2)
    os.makedirs(os.path.dirname(GEO_JSON), exist_ok=True)
    with open(GEO_JSON, "w", encoding="utf-8") as f:
        json.dump(geo_cache, f, ensure_ascii=False, indent=2)
else:
    with open(GEO_JSON, "r", encoding="utf-8") as f:
        geo_cache = json.load(f)
    logger.info(f"📦 좌표 캐시 사용 → {GEO_JSON}")

def get_station_num(region):
    info = geo_cache.get(region)
    if info and info.get("최근접관측소"):
        return info["최근접관측소"].get("지점번호")
    return None

hoqi_sum["지점번호"] = hoqi_sum["지역"].apply(get_station_num)
logger.info("📌 최근접 관측소 매핑 완료")
logger.info(f"   샘플:\n{hoqi_sum.head(3)}")

# ----------------------------------------------------
# 🔹 5. KMA 기상 데이터 병합
# ----------------------------------------------------
weather_files = sorted(glob(os.path.join(KMA_DIR, "OBS_ASOS_TIM_*.csv")))
weather_frames = []

for wf in tqdm(weather_files, desc="기상데이터 병합"):
    try:
        tmp = read_csv_safe(wf)
        tmp["일시"] = pd.to_datetime(tmp["일시"], errors="coerce")
        tmp = tmp[["지점","일시","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]].rename(columns={"지점":"지점번호"})
        weather_frames.append(tmp)
    except Exception as e:
        logger.warning(f"⚠️ 기상파일 '{os.path.basename(wf)}' 처리 실패: {e}")

df_weather = pd.concat(weather_frames, ignore_index=True).drop_duplicates(subset=["지점번호","일시"])
logger.info(f"📊 기상 데이터 총 {len(df_weather)}행 병합 완료")

# 병합 + null/"" → 0 처리
final_df = pd.merge(hoqi_sum, df_weather, how="left", on=["지점번호","일시"])
final_df[["합산발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]] = final_df[["합산발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]].fillna(0)

# 컬럼 순서 고정
final_df = final_df[["일시","발전구분","지역","지점번호","합산발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]]
final_df = final_df.sort_values(["지역","일시"]).reset_index(drop=True)
logger.info(f"✅ 최종 데이터 shape: {final_df.shape}")
logger.info(f"   샘플:\n{final_df.head(5)}")

# ----------------------------------------------------
# 🔹 6. 최종 CSV 저장
# ----------------------------------------------------
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
final_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
logger.info(f"✅ 최종 병합 완료 → {OUT_CSV}")



[2025-10-22 13:02:15,800]✅ INFO - 🗂 90개의 발전소 CSV 파일 수집됨


CSV 불러오기:   0%|          | 0/90 [00:00<?, ?it/s]

[2025-10-22 13:02:16,000]✅ INFO - 📄 '남동발전량_2022_01.csv' 읽기 성공 (utf-8)
[2025-10-22 13:02:16,045]✅ INFO - 📄 남동발전량_2022_01.csv 불러옴 (682행)
[2025-10-22 13:02:16,049]✅ INFO -    컬럼: ['발전구분', '호기', '일자', '1시 발전량(MWh)', '2시 발전량(MWh)', '3시 발전량(MWh)', '4시 발전량(MWh)', '5시 발전량(MWh)', '6시 발전량(MWh)', '7시 발전량(MWh)', '8시 발전량(MWh)', '9시 발전량(MWh)', '10시 발전량(MWh)', '11시 발전량(MWh)', '12시 발전량(MWh)', '13시 발전량(MWh)', '14시 발전량(MWh)', '15시 발전량(MWh)', '16시 발전량(MWh)', '17시 발전량(MWh)', '18시 발전량(MWh)', '19시 발전량(MWh)', '20시 발전량(MWh)', '21시 발전량(MWh)', '22시 발전량(MWh)', '23시 발전량(MWh)', '24시 발전량(MWh)', '총량(KW)', '평균(KW)', '최대(시간별)', '최소(시간별)', '최대', '최소']
[2025-10-22 13:02:16,080]✅ INFO -    샘플:
       발전구분  호기         일자  1시 발전량(MWh)  2시 발전량(MWh)  3시 발전량(MWh)  \
0     영동태양광   1 2022-01-31          0.0            0            0   
1  삼천포태양광#5   1 2022-01-31          0.0            0            0   

   4시 발전량(MWh)  5시 발전량(MWh)  6시 발전량(MWh)  7시 발전량(MWh)  ...  21시 발전량(MWh)  \
0            0            0            0         

C:\Users\baesa\AppData\Local\Temp\ipykernel_23424\4114509180.py:45: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
CSV 불러오기:   1%|          | 1/90 [00:00<00:13,  6.79it/s]

[2025-10-22 13:02:16,102]✅ INFO - 📄 '남동발전량_2022_02.csv' 읽기 성공 (utf-8)
[2025-10-22 13:02:16,110]✅ INFO - 📄 남동발전량_2022_02.csv 불러옴 (616행)
[2025-10-22 13:02:16,113]✅ INFO -    컬럼: ['발전구분', '호기', '일자', '1시 발전량(MWh)', '2시 발전량(MWh)', '3시 발전량(MWh)', '4시 발전량(MWh)', '5시 발전량(MWh)', '6시 발전량(MWh)', '7시 발전량(MWh)', '8시 발전량(MWh)', '9시 발전량(MWh)', '10시 발전량(MWh)', '11시 발전량(MWh)', '12시 발전량(MWh)', '13시 발전량(MWh)', '14시 발전량(MWh)', '15시 발전량(MWh)', '16시 발전량(MWh)', '17시 발전량(MWh)', '18시 발전량(MWh)', '19시 발전량(MWh)', '20시 발전량(MWh)', '21시 발전량(MWh)', '22시 발전량(MWh)', '23시 발전량(MWh)', '24시 발전량(MWh)', '총량(KW)', '평균(KW)', '최대(시간별)', '최소(시간별)', '최대', '최소']


...

[2025-10-22 13:02:35,434]✅ INFO - 📄 'OBS_ASOS_TIM_2024.csv' 읽기 성공 (cp949)


기상데이터 병합:  75%|███████▌  | 3/4 [00:07<00:02,  2.57s/it]

[2025-10-22 13:02:37,868]✅ INFO - 📄 'OBS_ASOS_TIM_2025.csv' 읽기 성공 (cp949)


기상데이터 병합: 100%|██████████| 4/4 [00:09<00:00,  2.41s/it]


[2025-10-22 13:02:39,873]✅ INFO - 📊 기상 데이터 총 3163359행 병합 완료
[2025-10-22 13:02:42,676]✅ INFO - ✅ 최종 데이터 shape: (474648, 9)
[2025-10-22 13:02:42,684]✅ INFO -    샘플:
                   일시   발전구분           지역  지점번호  합산발전량(MWh)  기온(°C)  강수량(mm)  \
0 2022-01-01 00:00:00  영동태양광  강원특별자치도 강릉시   105         0.0    -4.0      0.0   
1 2022-01-01 01:00:00  영동태양광  강원특별자치도 강릉시   105         0.0    -4.4      0.0   
2 2022-01-01 02:00:00  영동태양광  강원특별자치도 강릉시   105         0.0    -4.7      0.0   
3 2022-01-01 03:00:00  영동태양광  강원특별자치도 강릉시   105         0.0    -4.9      0.0   
4 2022-01-01 04:00:00  영동태양광  강원특별자치도 강릉시   105         0.0    -5.1      0.0   

   일조(hr)  일사(MJ/m2)  
0     0.0        0.0  
1     0.0        0.0  
2     0.0        0.0  
3     0.0        0.0  
4     0.0        0.0  
[2025-10-22 13:02:52,497]✅ INFO - ✅ 최종 병합 완료 → C:/ESG_Project1/file/merge_data\train_data.csv
